In [ ]:
!pip install Unidecode
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import pandas as pd
import json
import glob
import requests
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
import concurrent.futures
from bs4 import BeautifulSoup
from unidecode import unidecode

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Datos/ANID"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/ANID"

In [ ]:
df_anid = pd.read_excel(os.path.join(results, "data_anid_ruts_imputados.xlsx"))
df_anid

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,nombre_normalizado,rut_normalizado,rut_valido
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,65937660-1,True
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,CHILE,REGION DEL BIO-BIO,SCIA,UNIVERSIDAD DE CONCEPCION,81494400-k,True
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS -...,65997470-3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,FRANCIA,NaN,SREC,ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR...,-,False
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,FRANCIA,NaN,SREC,INRIA PARIS,-,False
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,ECUADOR,NaN,SREC,ESCUELA POLITECNICA NACIONAL,-,False
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,CHILE,NaN,SREC,FUNDACION INRIA CHILE,65058044-3,True


In [ ]:
df_anid["rut_sin_dv"] = df_anid["rut_normalizado"]
df_anid.loc[df_anid["rut_valido"], "rut_sin_dv"] = df_anid.loc[df_anid["rut_valido"], "rut_normalizado"].str.split("-").str[0]
df_anid

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,nombre_normalizado,rut_normalizado,rut_valido,rut_sin_dv
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False,-
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,65937660-1,True,65937660
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,CHILE,REGION DEL BIO-BIO,SCIA,UNIVERSIDAD DE CONCEPCION,81494400-k,True,81494400
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False,-
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS -...,65997470-3,True,65997470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,FRANCIA,NaN,SREC,ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR...,-,False,-
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,FRANCIA,NaN,SREC,INRIA PARIS,-,False,-
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,ECUADOR,NaN,SREC,ESCUELA POLITECNICA NACIONAL,-,False,-
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,CHILE,NaN,SREC,FUNDACION INRIA CHILE,65058044-3,True,65058044


# Descargar datos desde PortalChile

In [ ]:
def get_data_from_rut(rut):
  headers = {
    "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
  }
  url1 = "https://www.portalchile.org/api/suggest/{rut}?opcion=R"

  # La primera llamada retorna bastantes datos sobre la institución, pero presenta
  # un problema: en muchos casos, la actividad económica está incorrecta, ya que
  # sólo muestra la primera de ellas. Esto es especialmente problemático para universidades,
  # que suelen tener muchas actividades.
  # Por ello, de la primera respuesta sólo nos interesa obtener la razón social,
  # con la cual podemos hacer una segunda llamada a otro endpoint, que trae información
  # mucho más detallada.

  response1 = requests.get(url1.format(rut=rut), headers=headers)
  if response1.status_code != 200:
    return None

  suggestions = response1.json()["sugerencia"]
  if len(suggestions) != 1:
    return None

  data1 = response1.json()["sugerencia"][0]["_source"]
  rut_without_dv = data1["rut"]
  name_normalized = "-".join(data1["razon_social"].lower().split(" "))

  url2 = "https://www.portalchile.org/_next/data/yZqGI3zv6BcEg5khJvRzg/empresa/{name}-{rut}.json?id={name}-{rut}"
  response2 = requests.get(url2.format(name=name_normalized, rut=rut_without_dv), headers=headers)
  if response2.status_code != 200:
    return None
  return response2.json()



In [ ]:
def download_wrapper(rut):
  save_path = os.path.join(SAVE_DIR, f"{rut}.json")
  if os.path.exists(save_path):
    with open(save_path) as fp:
      return json.load(fp)
  data_rut = get_data_from_rut(rut)
  if data_rut is None:
    return

  with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(data_rut, f, ensure_ascii=False, indent=4)
  return data_rut

In [ ]:
ruts_unique = df_anid.loc[df_anid["rut_valido"], "rut_sin_dv"].unique()
len(ruts_unique)

1797

In [ ]:
common_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/Común"
SAVE_DIR = os.path.join(common_dir, "Instituciones")
os.makedirs(save_dir, exist_ok=True)

In [ ]:
data_institutions = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  futures = {
    executor.submit(download_wrapper, rut): rut
    for rut in ruts_unique
  }
  for future in tqdm(concurrent.futures.as_completed(futures)):
    rut = futures[future]
    data_institutions[rut] = future.result()

1797it [01:09, 25.68it/s]


## Construir dataframe

In [ ]:
data = []
for rut, institution in data_institutions.items():
  if institution is None:
    continue
  data.append([
    rut,
    institution["pageProps"]["data"]["empresa_rut"],
    institution["pageProps"]["data"]["empresa_nombre"],
    institution["pageProps"]["data"]["empresa_tipo"],
    institution["pageProps"]["data"]["empresa_subtipo"],
    institution["pageProps"]["data"]["empresa_provincia"],
    institution["pageProps"]["data"]["empresa_comuna"],
    institution["pageProps"]["data"]["actividades"],
    len(institution["pageProps"]["data"]["actividades"])
  ])

df_info = pd.DataFrame(data, columns=["rut", "rut_dv", "nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "actividades", "n_actividades"]).set_index("rut")
df_info

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades
rut,,,,,,,,
60908000,60908000-0,JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60908000', 'descripcion': 'Otras act...",2
65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '65937660', 'descripcion': 'Actividad...",2
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27
65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,"[{'rut': '65072516', 'descripcion': 'Investiga...",3
69508200,69508200-2,ORGANIZACIÓN DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,"[{'rut': '69508200', 'descripcion': 'Actividad...",1
...,...,...,...,...,...,...,...,...
65150812,65150812-6,FUNDACION PARQUE KATALAPI,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,"[{'rut': '65150812', 'descripcion': 'Fundacion...",2
69266010,69266010-2,ASOCIACION DE MUNICIPIOS RURALES REGIONES DE T...,MUNICIPALIDADES,OTRO ORG. MUNICIPAL,IQUIQUE,IQUIQUE,"[{'rut': '69266010', 'descripcion': 'Actividad...",2
65198382,65198382-7,FUNDACIÓN GEOGLIFOS DE TARAPACA,ORG. SIN FINES DE LUCRO,FUNDACION,TAMARUGAL,POZO ALMONTE,"[{'rut': '65198382', 'descripcion': 'Fundacion...",3


## Consolidar datos de actividades

Intentaremos dividir las instituciones en las siguientes categorías:
1. Instituciones de educación superior
2. Empresas (públicas/privadas)
3. Estado
4. IPSFL

In [ ]:
df_info["final"] = None

In [ ]:
unique_activities = {}
for rut, list_activities in df_info["actividades"].items():
  for single_activity in list_activities:
    id_activity = single_activity["idactividad"]
    if id_activity in unique_activities:
      unique_activities[id_activity]["instituciones"].append(rut)
    else:
      unique_activities[id_activity] = {
        "descripcion": single_activity["descripcion"],
        "instituciones": [rut]
      }

df_activities = pd.DataFrame.from_dict(unique_activities, orient="index")
df_activities

,descripcion,instituciones
879000,Otras actividades de atención en instituciones,"[60908000, 70377400, 61008000, 65045095, 72038..."
841100,Actividades de la administración pública en ge...,"[60908000, 61980230, 60915000, 72201100, 60605..."
803000,Actividades de investigación (incluye activida...,"[65937660, 81698900, 81494400, 71918700, 65076..."
829900,Otras actividades de servicios de apoyo a las ...,"[65937660, 71540100, 60910000, 70885500, 78158..."
681011,Alquiler de bienes inmuebles amoblados o con e...,"[81698900, 71133700, 70770800, 70754700, 70772..."
...,...,...
475100,"Venta al por menor de telas, lanas, hilos y si...",[65459510]
772100,Alquiler y arrendamiento de equipo recreativo ...,[65459510]
741002,Actividades de diseño y decoración de interiores,[65459510]
551009,Otras actividades de alojamiento para turistas...,[65459510]


### Instituciones de Educación Superior

#### Universidades

Para que una institución sea considerada universidad, debe:
1. Tener la actividad "Enseñanza superior en universidades públicas" o "Enseñanza superior en universidades públicas" (el substring "Enseñanza superior en universidades" sirve para ambos casos)
2. Tener la palabra _universidad_ en el nombre

In [ ]:
## df_activities.loc[...] return two rows, each one with a list of ruts.
## When we select .values, we end up with an array with two lists, so we merged them using itertools
ruts_universities = set(
    itertools.chain.from_iterable(
        df_activities.loc[df_activities["descripcion"].str.lower().str.contains("enseñanza superior en universidades"), "instituciones"].values
        )
    )
df_info.loc[(df_info.index.isin(ruts_universities)) &
            (df_info["nombre"].str.lower().str.contains("universidad")) &
            (df_info["final"].isnull()),
            "final"] = "Institución de Educación Superior"
df_info.loc[(df_info.index.isin(ruts_universities)) &
            (df_info["nombre"].str.lower().str.contains("universidad"))]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
87912900,87912900-1,UNIVERSIDAD DE LA FRONTERA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,"[{'rut': '87912900', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
81669200,81669200-8,PONTIFICIA UNIVERSIDAD CATOLICA DE VALPARAISO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81669200', 'descripcion': 'Suministr...",7,Institución de Educación Superior
71133700,71133700-8,UNIVERSIDAD DE MAGALLANES,ORG. SIN FINES DE LUCRO,OTRA OSFL,MAGALLANES,PUNTA ARENAS,"[{'rut': '71133700', 'descripcion': 'Alquiler ...",6,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
70770800,70770800-K,UNIVERSIDAD DE TARAPACA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ARICA,ARICA,"[{'rut': '70770800', 'descripcion': 'Compra, v...",3,Institución de Educación Superior
72012000,72012000-3,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,VITACURA,"[{'rut': '72012000', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior


#### Institutos Profesionales y Centros de Formación Técnica

In [ ]:
ruts_ips_cfts = set(itertools.chain.from_iterable(
    df_activities.loc[
        (df_activities["descripcion"].str.lower().str.contains("enseñanza superior en institutos profesionales")) |
        (df_activities["descripcion"].str.lower().str.contains("enseñanza superior en centros de formación")),
        "instituciones"].values)
    )
df_info.loc[(df_info.index.isin(ruts_ips_cfts)), ["nombre", "empresa_tipo", "empresa_subtipo", "n_actividades", "actividades"]]

,nombre,empresa_tipo,empresa_subtipo,n_actividades,actividades
rut,,,,,
72754700,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,1,"[{'rut': '72754700', 'descripcion': 'Enseñanza..."
70777500,UNIVERSIDAD ARTURO PRAT,ORG. SIN FINES DE LUCRO,OTRA OSFL,7,"[{'rut': '70777500', 'descripcion': 'Centros m..."
61103032,FUERZA AEREA DE CHILE COMANDANCIA EN JEFE COMA...,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,6,"[{'rut': '61103032', 'descripcion': 'Enseñanza..."
70208100,FUNDACION LA FRONTERA,ORG. SIN FINES DE LUCRO,OTRA OSFL,2,"[{'rut': '70208100', 'descripcion': 'Otras act..."
96930530,VCL S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,5,"[{'rut': '96930530', 'descripcion': 'Venta al ..."
70859400,CORP MUNICIPAL DE VALPARAISO PARA EL DESARROLL...,ORG. SIN FINES DE LUCRO,OTRA OSFL,9,"[{'rut': '70859400', 'descripcion': 'Compra, v..."
72837400,COOP CAMPESINA APICOLA SANTA BARBA RA LTDA,ORG. SIN FINES DE LUCRO,COOPERATIVA,6,"[{'rut': '72837400', 'descripcion': 'Enseñanza..."
69190900,I MUNICIPALIDAD DE FREIRE,MUNICIPALIDADES,MUNICIPALIDAD,11,"[{'rut': '69190900', 'descripcion': 'Suministr..."
96544210,CORPORACION EDUCACIONAL VIRGINIO GOMEZ,ORG. SIN FINES DE LUCRO,CORPORACION,2,"[{'rut': '96544210', 'descripcion': 'Alquiler ..."


Son IPs o CFTs:
- IACC S.A.
- CORPORACION EDUCACIONAL VIRGINIO GOMEZ
- CORPORACION CENTRO DE FORMACION TECNICA DE LA PONTIFICIA UNIVERSIDAD CATOLICA DE
- FUND INSTITUTO PROFESIONAL DUOC UC
- CENTRO DE FORMACIÓN TÉCNICA ESTATAL REGIÓN ARICA Y PARINACOTA
- CENTRO DE FORMACION TECNICA DE LA REGION DE TARAPACA

In [ ]:
df_info.loc[["87845200", "96544210", "99538520", "72754700", "53333710", "65154398"], "final"] = "Institución de Educación Superior"
df_info.loc[["87845200", "96544210", "99538520", "72754700", "53333710", "65154398"]]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
96544210,96544210-3,CORPORACION EDUCACIONAL VIRGINIO GOMEZ,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '96544210', 'descripcion': 'Alquiler ...",2,Institución de Educación Superior
99538520,99538520-1,CORPORACION CENTRO DE FORMACION TECNICA DE LA ...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,"[{'rut': '99538520', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
53333710,53333710-4,CENTRO DE FORMACIÓN TÉCNICA ESTATAL REGIÓN ARI...,ORG. SIN FINES DE LUCRO,OTRA OSFL,ARICA,ARICA,"[{'rut': '53333710', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
65154398,65154398-3,CENTRO DE FORMACION TECNICA DE LA REGION DE TA...,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,IQUIQUE,IQUIQUE,"[{'rut': '65154398', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior


In [ ]:
df_info.loc[df_info["final"] == "Institución de Educación Superior"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
87912900,87912900-1,UNIVERSIDAD DE LA FRONTERA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,"[{'rut': '87912900', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
81669200,81669200-8,PONTIFICIA UNIVERSIDAD CATOLICA DE VALPARAISO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81669200', 'descripcion': 'Suministr...",7,Institución de Educación Superior
71133700,71133700-8,UNIVERSIDAD DE MAGALLANES,ORG. SIN FINES DE LUCRO,OTRA OSFL,MAGALLANES,PUNTA ARENAS,"[{'rut': '71133700', 'descripcion': 'Alquiler ...",6,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
70770800,70770800-K,UNIVERSIDAD DE TARAPACA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ARICA,ARICA,"[{'rut': '70770800', 'descripcion': 'Compra, v...",3,Institución de Educación Superior
72012000,72012000-3,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,VITACURA,"[{'rut': '72012000', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior


### Empresas

Consideraremos empresas privadas y públicas


#### Empresas públicas

In [ ]:
df_soc = pd.read_excel(os.path.join(common_dir, "data_empresas_estatales.xlsx"))
df_soc["rut_sin_dv"] = df_soc["rut_sin_dv"].astype(str)
df_soc

,nombre,rut,rut_sin_dv
0,Empresa Portuaria Arica,61945700-5,61945700
1,Empresa Portuaria Iquique,61951300-2,61951300
2,Empresa Portuaria Antofagasta,73968300-9,73968300
3,Empresa Portuaria Coquimbo,61946300-5,61946300
4,Empresa Portuaria Valparaíso,61952700-3,61952700
5,Empresa Portuaria San Antonio,61960100-9,61960100
6,Empresa Portuaria Talcahuano San Vicente,61960300-1,61960300
7,Empresa Portuaria Puerto Montt,61950900-5,61950900
8,Empresa Portuaria Chacabuco,61959100-3,61959100
9,Empresa Portuaria Austral,61956700-5,61956700


Veamos ahora si hay empresas públicas en nuestro conjunto de datos

In [ ]:
df_info[df_info.index.isin(df_soc["rut_sin_dv"].values)]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
61216000,61216000-7,EMPRESA DE LOS FERROCARRILES DEL ESTADO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61216000', 'descripcion': 'Alquiler ...",3,None
73968300,73968300-9,EMPRESA PORTUARIA ANTOFAGASTA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '73968300', 'descripcion': 'Otros ser...",2,None
61219000,61219000-3,EMPRESA DE TRANSPORTE DE PASAJEROS METRO S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,SANTIAGO,"[{'rut': '61219000', 'descripcion': 'Compra, v...",5,None


Efectivamente, hay tres empresas públicas: EFE, Empresa Portuaria Antofagasta y Metro. A estas tres instituciones les asignaremos la clasificación _Empresa_. Nos fijamos además que tienen tipo y subtipo distinto, lo cual demuestra que no era trivial caracterizar este tipo de empresas a partir de los datos.

In [ ]:
df_info.loc[df_info.index.isin(df_soc["rut_sin_dv"].values), "final"] = "Empresa"
df_info.loc[df_info.index.isin(df_soc["rut_sin_dv"].values)]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
61216000,61216000-7,EMPRESA DE LOS FERROCARRILES DEL ESTADO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61216000', 'descripcion': 'Alquiler ...",3,Empresa
73968300,73968300-9,EMPRESA PORTUARIA ANTOFAGASTA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '73968300', 'descripcion': 'Otros ser...",2,Empresa
61219000,61219000-3,EMPRESA DE TRANSPORTE DE PASAJEROS METRO S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,SANTIAGO,"[{'rut': '61219000', 'descripcion': 'Compra, v...",5,Empresa


#### Empresas privadas

Empresas privadas serán todas aquellas que tengan tipo _PERSONA JURIDICA COMERCIAL_. De todas formas, primero revisaremos si existen instituciones con ese tipo y a las que ya le hayamos asignado clasificación.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL") & (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81518400,81518400-9,UNIVERSIDAD CATOLICA DEL NORTE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81518400', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
61219000,61219000-3,EMPRESA DE TRANSPORTE DE PASAJEROS METRO S A,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,SANTIAGO,"[{'rut': '61219000', 'descripcion': 'Compra, v...",5,Empresa


Existen 3 instituciones con tipo _PERSONA JURIDICA COMERCIAL_ que clasificamos como Instituciones de Educación Superior, y una que está clasificada commo Empresa (Metro). Debemos tener cuidado de no sobreescribir esa clasificación previa.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL") &
            (df_info["final"].isnull()),
            "final"] = "Empresa"
df_info.loc[(df_info["empresa_tipo"] == "PERSONA JURIDICA COMERCIAL")]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81518400,81518400-9,UNIVERSIDAD CATOLICA DEL NORTE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81518400', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
78158620,78158620-K,CENTRO DE INFORMACION TECNOLOGICA LIMITA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ELQUI,LA SERENA,"[{'rut': '78158620', 'descripcion': 'Otras act...",9,Empresa
76230007,76230007-9,COMERCIAL AGROPROCESO SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CAUTÍN,LAUTARO,"[{'rut': '76230007', 'descripcion': 'Transport...",5,Empresa
96689640,96689640-K,INSTITUTO TECNOLOGICO DEL SALMON SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,LLANQUIHUE,PUERTO VARAS,"[{'rut': '96689640', 'descripcion': 'Investiga...",1,Empresa
...,...,...,...,...,...,...,...,...,...
76299833,76299833-5,SOCIEDAD DE PROFESIONALES ARANEDA TONELLI MORE...,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,CACHAPOAL,RANCAGUA,"[{'rut': '76299833', 'descripcion': 'Otras act...",1,Empresa
76369697,76369697-9,CAPRILAC SOCIEDAD GANADERA LIMITADA,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,LAS CONDES,"[{'rut': '76369697', 'descripcion': 'Cultivo d...",4,Empresa
77134472,77134472-0,QUESERIAS DE MARCHIGÜE SPA,PERSONA JURIDICA COMERCIAL,SOCIEDAD POR ACCIONES,CARDENAL CARO,MARCHIGUE,"[{'rut': '77134472', 'descripcion': 'Elaboraci...",1,Empresa


Revisaremos no haber sobreescrito las IES con tipo _PERSONA JURIDICA COMERCIAL_.

In [ ]:
df_info.loc[["87845200", "81698900", "81518400"]]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
87845200,87845200-3,IACC S.A.,PERSONA JURIDICA COMERCIAL,SOCIEDADES ANONIMAS CERRADAS,SANTIAGO,PROVIDENCIA,"[{'rut': '87845200', 'descripcion': 'Enseñanza...",3,Institución de Educación Superior
81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,"[{'rut': '81698900', 'descripcion': 'Alquiler ...",27,Institución de Educación Superior
81518400,81518400-9,UNIVERSIDAD CATOLICA DEL NORTE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '81518400', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior


### Estado

Asignaremos a Estado todas las instituciones que cuenten con las siguientes características:
1. Tener tipo _INSTITUCIONES FISCALES_ O _MUNICIPALIDADES_
2. No tener clasificación previa asignada.

Primero, revisaremos qué instituciones tienen tipo _INSITUCIONES FISCALES_ o _MUNICIPALIDAD_ y SÍ tienen clasificación previa.

In [ ]:
df_info[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"])) &
        (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
87912900,87912900-1,UNIVERSIDAD DE LA FRONTERA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,"[{'rut': '87912900', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
60921000,60921000-1,UNIVERSIDAD DE VALPARAISO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,VALPARAÍSO,VALPARAISO,"[{'rut': '60921000', 'descripcion': 'Compra, v...",12,Institución de Educación Superior
60911000,60911000-7,UNIVERSIDAD DE SANTIAGO DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,EST CENTRAL,"[{'rut': '60911000', 'descripcion': 'Compra, v...",11,Institución de Educación Superior
61980520,61980520-8,UNIVERSIDAD DE AYSEN,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,COIHAIQUE,COYHAIQUE,"[{'rut': '61980520', 'descripcion': 'Compra, v...",2,Institución de Educación Superior
61980530,61980530-5,UNIVERSIDAD DE OHIGGINS,INSTITUCIONES FISCALES,ORG. EDUCACION SUPERIOR,CACHAPOAL,RANCAGUA,"[{'rut': '61980530', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60911006,60911006-6,UNIVERSIDAD DEL BIO BIO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CONCEPCIÓN,CONCEPCION,"[{'rut': '60911006', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60910047,60910047-8,UNIVERSIDAD METROPOLITANA DE CIENCIAS DE LA ED...,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,NUNOA,"[{'rut': '60910047', 'descripcion': 'Enseñanza...",5,Institución de Educación Superior
61216000,61216000-7,EMPRESA DE LOS FERROCARRILES DEL ESTADO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61216000', 'descripcion': 'Alquiler ...",3,Empresa


Nuevamente, debemos tener cuidado de no sobreeescribir esas instituciones.

In [ ]:
df_info.loc[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"])) &
            (df_info["final"].isnull()),
            "final"] = "Estado"
df_info.loc[(df_info["empresa_tipo"].isin(["INSTITUCIONES FISCALES", "MUNICIPALIDADES"]))]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
60908000,60908000-0,JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60908000', 'descripcion': 'Otras act...",2,Estado
61980230,61980230-6,AGENCIA DE CALIDAD DE LA EDUCACION,INSTITUCIONES FISCALES,ORGANISMO AUTONOMO DEL ESTADO,SANTIAGO,SANTIAGO,"[{'rut': '61980230', 'descripcion': 'Actividad...",2,Estado
87912900,87912900-1,UNIVERSIDAD DE LA FRONTERA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,"[{'rut': '87912900', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
60910000,60910000-1,UNIVERSIDAD DE CHILE,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '60910000', 'descripcion': 'Enseñanza...",4,Institución de Educación Superior
60921000,60921000-1,UNIVERSIDAD DE VALPARAISO,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,VALPARAÍSO,VALPARAISO,"[{'rut': '60921000', 'descripcion': 'Compra, v...",12,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
61607901,61607901-8,HOSPITAL CLINICO DE MAGALLANES DR. LAUTARO NAV...,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,MAGALLANES,PUNTA ARENAS,"[{'rut': '61607901', 'descripcion': 'Actividad...",1,Estado
61608602,61608602-2,HOSPITAL DE URGENCIA ASISTENCIA PUBLICA DR ALE...,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,"[{'rut': '61608602', 'descripcion': 'Centros m...",2,Estado
69091300,69091300-3,I MUNICIPALIDAD DE MARCHIGUE,MUNICIPALIDADES,MUNICIPALIDAD,CARDENAL CARO,MARCHIGUE,"[{'rut': '69091300', 'descripcion': 'Centros m...",6,Estado


Vemos que en la etiqueta _final_ aparece tanto Estado como Institución de Educación Superior, indicando que los datos previos no se sobreescribieron.

### Instituciones Privadas Sin Fines de Lucro (IPSFL)

Veamos primero las organizaciones que aún no tienen clasificación

In [ ]:
df_info.loc[df_info["final"].isnull()].groupby(["empresa_tipo", "empresa_subtipo"])["nombre"].count()

empresa_tipo                empresa_subtipo                   
ORG. SIN FINES DE LUCRO     ASOC. GREMIAL                          19
                            COOPERATIVA                            14
                            CORPORACION                            51
                            FUNDACION                              55
                            JUNTA DE VECINOS, ORG. COMUNITARIA      6
                            OTRA OSFL                             191
                            SINDICATO                               1
ORGANISMOS INTERNACIONALES  ORGANISMOS INTERNACIONALES              3
SIN PER. JURIDICA           SOCIEDADES DE HECHO                     1
                            SUCESIONES O COMUNIDADES HERED          2
SOCIEDADES EXTRANJERAS      AGENCIA                                 2
                            ENTIDAD SIN RESIDENCIA                  2
Name: nombre, dtype: int64

Revisemos las instituciones con tipo _ORG. SIN FINES DE LUCRO_

In [ ]:
df_info.loc[df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '65937660', 'descripcion': 'Actividad...",2,None
65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,"[{'rut': '65072516', 'descripcion': 'Investiga...",3,None
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
65997470,65997470-3,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS,ORG. SIN FINES DE LUCRO,OTRA OSFL,ELQUI,LA SERENA,"[{'rut': '65997470', 'descripcion': 'Investiga...",1,None
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
65459510,65459510-0,FUND HUILO HUILO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALDIVIA,PANGUIPULLI,"[{'rut': '65459510', 'descripcion': 'Venta al ...",29,None
70866900,70866900-8,SOC CHILENA DE NUTRICION BROMATOLOGIA Y TOXICO...,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '70866900', 'descripcion': 'Actividad...",3,None
65150812,65150812-6,FUNDACION PARQUE KATALAPI,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,"[{'rut': '65150812', 'descripcion': 'Fundacion...",2,None


Como siempre, debemos tener cuidado con las instituciones con clasificación previa.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO") &
            (~df_info["final"].isnull())]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
81669200,81669200-8,PONTIFICIA UNIVERSIDAD CATOLICA DE VALPARAISO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALPARAÍSO,VALPARAISO,"[{'rut': '81669200', 'descripcion': 'Suministr...",7,Institución de Educación Superior
71133700,71133700-8,UNIVERSIDAD DE MAGALLANES,ORG. SIN FINES DE LUCRO,OTRA OSFL,MAGALLANES,PUNTA ARENAS,"[{'rut': '71133700', 'descripcion': 'Alquiler ...",6,Institución de Educación Superior
71540100,71540100-2,UNIVERSIDAD NACIONAL ANDRES BELLO,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,SANTIAGO,"[{'rut': '71540100', 'descripcion': 'Enseñanza...",2,Institución de Educación Superior
70770800,70770800-K,UNIVERSIDAD DE TARAPACA,ORG. SIN FINES DE LUCRO,OTRA OSFL,ARICA,ARICA,"[{'rut': '70770800', 'descripcion': 'Compra, v...",3,Institución de Educación Superior
72012000,72012000-3,UNIVERSIDAD TECNOLOGICA DE CHILE INACAP,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,VITACURA,"[{'rut': '72012000', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
81380500,81380500-6,UNIVERSIDAD AUSTRAL DE CHILE,ORG. SIN FINES DE LUCRO,CORPORACION,VALDIVIA,VALDIVIA,"[{'rut': '81380500', 'descripcion': 'Suministr...",7,Institución de Educación Superior
71918700,71918700-5,UNIVERSIDAD CATOLICA DE TEMUCO,ORG. SIN FINES DE LUCRO,OTRA OSFL,CAUTÍN,TEMUCO,"[{'rut': '71918700', 'descripcion': 'Enseñanza...",14,Institución de Educación Superior


Aparecen IES y una empresa estatal; no debemos sobreescribirlas.

In [ ]:
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO") &
            (df_info["final"].isnull()),
            "final"] = "Institución Privada Sin Fines de Lucro"
df_info.loc[(df_info["empresa_tipo"] == "ORG. SIN FINES DE LUCRO")]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,"[{'rut': '65937660', 'descripcion': 'Actividad...",2,Institución Privada Sin Fines de Lucro
65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,"[{'rut': '65072516', 'descripcion': 'Investiga...",3,Institución Privada Sin Fines de Lucro
72754700,72754700-2,FUND INSTITUTO PROFESIONAL DUOC UC,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '72754700', 'descripcion': 'Enseñanza...",1,Institución de Educación Superior
65997470,65997470-3,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS,ORG. SIN FINES DE LUCRO,OTRA OSFL,ELQUI,LA SERENA,"[{'rut': '65997470', 'descripcion': 'Investiga...",1,Institución Privada Sin Fines de Lucro
81494400,81494400-K,UNIVERSIDAD DE CONCEPCION,ORG. SIN FINES DE LUCRO,CORPORACION,CONCEPCIÓN,CONCEPCION,"[{'rut': '81494400', 'descripcion': 'Suministr...",31,Institución de Educación Superior
...,...,...,...,...,...,...,...,...,...
65459510,65459510-0,FUND HUILO HUILO,ORG. SIN FINES DE LUCRO,OTRA OSFL,VALDIVIA,PANGUIPULLI,"[{'rut': '65459510', 'descripcion': 'Venta al ...",29,Institución Privada Sin Fines de Lucro
70866900,70866900-8,SOC CHILENA DE NUTRICION BROMATOLOGIA Y TOXICO...,ORG. SIN FINES DE LUCRO,OTRA OSFL,SANTIAGO,PROVIDENCIA,"[{'rut': '70866900', 'descripcion': 'Actividad...",3,Institución Privada Sin Fines de Lucro
65150812,65150812-6,FUNDACION PARQUE KATALAPI,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,"[{'rut': '65150812', 'descripcion': 'Fundacion...",2,Institución Privada Sin Fines de Lucro


Aparecen tanto IPSFL como IES, lo cual muestra que no hubo sobreescritura.

### Otras instituciones

Veamos las instituciones que hasta ahora no han sido categorizadas

In [ ]:
df_info[df_info["final"].isnull()]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
69508200,69508200-2,ORGANIZACIÓN DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,"[{'rut': '69508200', 'descripcion': 'Actividad...",1,None
59281960,59281960-0,LABORELEC LATIN AMERICA,SOCIEDADES EXTRANJERAS,AGENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59281960', 'descripcion': 'Venta al ...",2,None
69500700,69500700-0,AGENCIA DE COOPERACION INTERNACIONAL DEL JAPON,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,LAS CONDES,[],0,None
86160300,86160300-8,BHP CHILE INC,SOCIEDADES EXTRANJERAS,AGENCIA,SANTIAGO,LAS CONDES,"[{'rut': '86160300', 'descripcion': 'Compra, v...",5,None
59087530,59087530-9,ECOMETALES LIMITED AGENCIA EN CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59087530', 'descripcion': 'Recuperac...",2,None
53323948,53323948-K,SUCESION MARIA ENEDINA BARRIA HERNANDEZ,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,CHILOÉ,ANCUD,"[{'rut': '53323948', 'descripcion': 'Venta al ...",2,None
50213140,50213140-0,GALFANO Y COMPANIA LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,PENALOLEN,"[{'rut': '50213140', 'descripcion': 'Otras act...",2,None
53194330,53194330-9,SUC ADOLFO MIGUEL CASCARDO OSORIO,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,MAGALLANES,PUNTA ARENAS,"[{'rut': '53194330', 'descripcion': 'Cría de o...",1,None
59102410,59102410-8,WOODLAND DEVELOPMENT COMPANY LLC,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,CORRAL,"[{'rut': '59102410', 'descripcion': 'Extracció...",2,None


Las sociedades extranjeras y los organismos internacionales los dejaremos como "Internacional".

In [ ]:
df_info.loc[(df_info["empresa_tipo"].isin(["SOCIEDADES EXTRANJERAS", "ORGANISMOS INTERNACIONALES"])) &
            (df_info["final"].isnull()),
            "final"] = "Internacional"
df_info.loc[df_info["final"] == "Internacional"]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
69508200,69508200-2,ORGANIZACIÓN DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,"[{'rut': '69508200', 'descripcion': 'Actividad...",1,Internacional
59281960,59281960-0,LABORELEC LATIN AMERICA,SOCIEDADES EXTRANJERAS,AGENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59281960', 'descripcion': 'Venta al ...",2,Internacional
69500700,69500700-0,AGENCIA DE COOPERACION INTERNACIONAL DEL JAPON,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,LAS CONDES,[],0,Internacional
86160300,86160300-8,BHP CHILE INC,SOCIEDADES EXTRANJERAS,AGENCIA,SANTIAGO,LAS CONDES,"[{'rut': '86160300', 'descripcion': 'Compra, v...",5,Internacional
59087530,59087530-9,ECOMETALES LIMITED AGENCIA EN CHILE,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,PROVIDENCIA,"[{'rut': '59087530', 'descripcion': 'Recuperac...",2,Internacional
59102410,59102410-8,WOODLAND DEVELOPMENT COMPANY LLC,SOCIEDADES EXTRANJERAS,ENTIDAD SIN RESIDENCIA,SANTIAGO,CORRAL,"[{'rut': '59102410', 'descripcion': 'Extracció...",2,Internacional
69500400,69500400-1,INSTITUTO INTERAMERICANO DE COOPERACION PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,"[{'rut': '69500400', 'descripcion': 'Actividad...",1,Internacional


Veamos ahora los casos restantes

In [ ]:
df_info.loc[df_info["final"].isnull()]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
53323948,53323948-K,SUCESION MARIA ENEDINA BARRIA HERNANDEZ,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,CHILOÉ,ANCUD,"[{'rut': '53323948', 'descripcion': 'Venta al ...",2,None
50213140,50213140-0,GALFANO Y COMPANIA LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,PENALOLEN,"[{'rut': '50213140', 'descripcion': 'Otras act...",2,None
53194330,53194330-9,SUC ADOLFO MIGUEL CASCARDO OSORIO,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,MAGALLANES,PUNTA ARENAS,"[{'rut': '53194330', 'descripcion': 'Cría de o...",1,None


_Galfano y compañía limitada_ parecen ser una empresa, al menos según https://www.solomira.com/explorar/vendor/mPRpayt75fr5 . Las otras dos las dejaremos como _Otra_, ya que no hay claridad aún respecto a cómo clasificarlas.

In [ ]:
df_info.loc["50213140", "final"] = "Empresa"
df_info.loc[["53194330", "53323948"], "final"] = "Otra"
df_info.loc[["50213140", "53194330", "53323948"]]

,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,actividades,n_actividades,final
rut,,,,,,,,,
50213140,50213140-0,GALFANO Y COMPANIA LIMITADA,SIN PER. JURIDICA,SOCIEDADES DE HECHO,SANTIAGO,PENALOLEN,"[{'rut': '50213140', 'descripcion': 'Otras act...",2,Empresa
53194330,53194330-9,SUC ADOLFO MIGUEL CASCARDO OSORIO,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,MAGALLANES,PUNTA ARENAS,"[{'rut': '53194330', 'descripcion': 'Cría de o...",1,Otra
53323948,53323948-K,SUCESION MARIA ENEDINA BARRIA HERNANDEZ,SIN PER. JURIDICA,SUCESIONES O COMUNIDADES HERED,CHILOÉ,ANCUD,"[{'rut': '53323948', 'descripcion': 'Venta al ...",2,Otra


In [ ]:
df_info["final"].value_counts()

Empresa                                   1051
Institución Privada Sin Fines de Lucro     337
Estado                                     240
Institución de Educación Superior           55
Internacional                                7
Otra                                         2
Name: final, dtype: int64

In [ ]:
df_final = df_info.rename(columns={"final": "tipo_institucion_final"})[["rut_dv", "nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "tipo_institucion_final"]].reset_index()
df_final

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,60908000,60908000-0,JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,Estado
1,65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,Institución Privada Sin Fines de Lucro
2,81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,Institución de Educación Superior
3,65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,Institución Privada Sin Fines de Lucro
4,69508200,69508200-2,ORGANIZACIÓN DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,Internacional
...,...,...,...,...,...,...,...,...
1687,65150812,65150812-6,FUNDACION PARQUE KATALAPI,ORG. SIN FINES DE LUCRO,FUNDACION,SANTIAGO,LAS CONDES,Institución Privada Sin Fines de Lucro
1688,69266010,69266010-2,ASOCIACION DE MUNICIPIOS RURALES REGIONES DE T...,MUNICIPALIDADES,OTRO ORG. MUNICIPAL,IQUIQUE,IQUIQUE,Estado
1689,65198382,65198382-7,FUNDACIÓN GEOGLIFOS DE TARAPACA,ORG. SIN FINES DE LUCRO,FUNDACION,TAMARUGAL,POZO ALMONTE,Institución Privada Sin Fines de Lucro
1690,72396000,72396000-2,CORP NACIONAL DE DESARROLLO INDIGENA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,CAUTÍN,TEMUCO,Estado


# Descargar datos desde Registros 19862

In [ ]:
missing_ruts = set(df_anid["rut_sin_dv"].unique()) - set(df_final["rut"])
missing_ruts

{'-',
 '10105973',
 '12345678',
 '12695861',
 '13109718',
 '14439042',
 '15447109',
 '15960567',
 '16474185',
 '1767',
 '18019651',
 '20340775',
 '4860556',
 '53307123',
 '53325564',
 '53333021',
 '55555555',
 '59163860',
 '59203220',
 '60301000',
 '60701004',
 '61312010',
 '61602054',
 '61602189',
 '6187519',
 '61937800',
 '64096640',
 '65001733',
 '65016972',
 '65034388',
 '65035014',
 '65054207',
 '65059612',
 '65064656',
 '65067859',
 '65090383',
 '65093804',
 '65119168',
 '65123393',
 '65124334',
 '65151574',
 '65152444',
 '65157826',
 '65160115',
 '65167516',
 '65168136',
 '65170251',
 '65173837',
 '65184889',
 '65188109',
 '65189901',
 '65191034',
 '65200667',
 '65205033',
 '65205332',
 '65205548',
 '65207282',
 '65207445',
 '65209800',
 '65211227',
 '65212859',
 '65382830',
 '65454820',
 '65542390',
 '65667480',
 '65886860',
 '65904370',
 '65920580',
 '65936650',
 '69160700-',
 '69500502',
 '69500900',
 '70037600',
 '70708600',
 '72299300',
 '72319400',
 '72847600',
 '76060484'

In [ ]:
def get_data_from_registros1982(rut):
  url = f"https://registros19862.gob.cl/institucion/{rut}/mas"
  response = requests.get(url)
  if response.status_code != 200:
    return None
  result = {}
  soup = BeautifulSoup(response.text)
  payload = soup.find("section", class_="payload")
  name, type_header = [txt.strip() for txt in payload.find("div", class_="title").h1.text.split("\n") if txt.strip() != ""]
  result["Nombre"] = name
  result["Tipo 1"] = type_header
  rows = payload.find("table").find_all("tr")
  for row in rows:
    attr_name = row.th.text.strip()
    value = row.td.text.strip()
    result[attr_name] = value
  return result

In [ ]:
results = {}
for rut in tqdm(missing_ruts):
  data = get_data_from_registros1982(rut)
  if data is None:
    print(f"RUT {rut} no encontrado")
    continue
  results[rut] = data

  1%|          | 1/107 [00:00<01:17,  1.37it/s]

RUT 59203220 no encontrado


  3%|▎         | 3/107 [00:01<01:03,  1.63it/s]

RUT 8104790 no encontrado


  4%|▎         | 4/107 [00:02<01:02,  1.65it/s]

...

RUT 65191034 no encontrado


 98%|█████████▊| 105/107 [01:05<00:01,  1.63it/s]

RUT 65167516 no encontrado


100%|██████████| 107/107 [01:07<00:00,  1.59it/s]

RUT 53333021 no encontrado


## Construir dataframe

In [ ]:
df_data = pd.DataFrame.from_dict(results, orient="index")
df_data

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros
65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30
55555555,PRUEBA DE INSCRIPCIÓN INSCRIPCIÓN EN LA LEY,INSTITUCIÓN PRIVADA,55.555.555-5,ONG,"Informática (computación, internet)",34 otorgada por Mi,$ 0,$ 0,$ 0,Representante legal\nYASNA CAROLINA ROJAS RIVE...,SALITRERA SEBASTOPOL 2044\n ...,23-09-2005 15:56
65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26
65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38
77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43
4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21
77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10
65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52


## Consolidar datos de actividades

Nuevamente, los dividiremos en
1. Instituciones de Educación Superior
2. Empresas
3. Estado
4. IPSFL
5. Otras

#### Instituciones de Educación Superior

Dado que son pocos datos, podemos revisarlos manualmente. Así, nos fijamos que no hay ninguno que corresponda a una IES, por lo cual no hay nada que hacer aquí

#### Empresas

In [ ]:
df_data["Tipo institución"].unique()

array(['Asociaciones', 'ONG',
       'Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)',
       'Corporaciones (Ej: Educación, Salud, Cultura, etc.)', 'Empresas',
       'Agrupaciones',
       'Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.)',
       'Organismo Internacional',
       'Uniones comunales de organizaciones funcionales', 'Comités', nan],
      dtype=object)

Caracterizaremos como empresas a todas aquellas instituciones que tengan el valor `Empresas` en la columna `Tipo institución`

In [ ]:
df_data["tipo_institucion_final"] = None
df_data.loc[df_data["Tipo institución"] == "Empresas", "tipo_institucion_final"] = "Empresa"
df_data

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,None
55555555,PRUEBA DE INSCRIPCIÓN INSCRIPCIÓN EN LA LEY,INSTITUCIÓN PRIVADA,55.555.555-5,ONG,"Informática (computación, internet)",34 otorgada por Mi,$ 0,$ 0,$ 0,Representante legal\nYASNA CAROLINA ROJAS RIVE...,SALITRERA SEBASTOPOL 2044\n ...,23-09-2005 15:56,None
65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,None
65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,None
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,None
77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43,Empresa
4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,None
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,None
77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10,Empresa
65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,None


#### Estado

Parece haber sólo una institución que es servicio público.

In [ ]:
df_data[df_data["Tipo 1"] == "SERVICIO PÚBLICO"]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
61602189,HOSPITAL GUILLERMO GRANT BENAVENTE,SERVICIO PÚBLICO,61.602.189-3,NaN,NaN,NaN,NaN,NaN,NaN,Representante legal\nCARLOS ALBERTO CAPURRO DU...,SAN MARTIN 1436\n ...,24-10-2018 19:00,None


In [ ]:
df_data.loc[df_data["Tipo 1"] == "SERVICIO PÚBLICO", "tipo_institucion_final"] = "Estado"

In [ ]:
df_data

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,None
55555555,PRUEBA DE INSCRIPCIÓN INSCRIPCIÓN EN LA LEY,INSTITUCIÓN PRIVADA,55.555.555-5,ONG,"Informática (computación, internet)",34 otorgada por Mi,$ 0,$ 0,$ 0,Representante legal\nYASNA CAROLINA ROJAS RIVE...,SALITRERA SEBASTOPOL 2044\n ...,23-09-2005 15:56,None
65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,None
65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,None
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,None
77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43,Empresa
4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,None
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,None
77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10,Empresa
65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,None


#### IPSFL

Serán todas aquellas que tengan el tipo:
- Agrupaciones
- ONG
- Asociaciones
- Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)
- Comités
- Corporaciones (Ej: Educación, Salud, Cultura, etc.)
- Uniones comunales de organizaciones funcionales
- Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.),


In [ ]:
ipsfl_types = ['Agrupaciones', 'ONG', 'Asociaciones',
               'Fundaciones (Ej: CEMA Chile, Integra, Cruz Roja, Scout, etc.)',
               'Comités', 'Corporaciones (Ej: Educación, Salud, Cultura, etc.)',
               'Uniones comunales de organizaciones funcionales',
               'Centros (Ej: Madres, Padres, Apoderados, Alumnos, Estudio, Investigación, etc.)']
df_data.loc[df_data["Tipo institución"].isin(ipsfl_types), "tipo_institucion_final"] = "Institución Privada Sin Fines de Lucro"
df_data.loc[df_data["Tipo institución"].isin(ipsfl_types)]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,Institución Privada Sin Fines de Lucro
55555555,PRUEBA DE INSCRIPCIÓN INSCRIPCIÓN EN LA LEY,INSTITUCIÓN PRIVADA,55.555.555-5,ONG,"Informática (computación, internet)",34 otorgada por Mi,$ 0,$ 0,$ 0,Representante legal\nYASNA CAROLINA ROJAS RIVE...,SALITRERA SEBASTOPOL 2044\n ...,23-09-2005 15:56,Institución Privada Sin Fines de Lucro
65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,Institución Privada Sin Fines de Lucro
65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,Institución Privada Sin Fines de Lucro
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,Institución Privada Sin Fines de Lucro
4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,Institución Privada Sin Fines de Lucro
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,Institución Privada Sin Fines de Lucro
65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,Institución Privada Sin Fines de Lucro
65123393,AGRUPACION DE PRODUCTORES OVINOS DE MAFIL,INSTITUCIÓN PRIVADA,65.123.393-3,Agrupaciones,Apoyo y fomento productivo,ley 19.418 otorgada por municipalidad de mafil,$ 0,$ 0,$ 0,Representante legal\nLAURA SANTANA ZIEBALLA\n5...,LOS AGUILA\n ...,30-06-2017 11:49,Institución Privada Sin Fines de Lucro
65034388,SINDICATO DE TRABAJADORES INDEPENDIENTES RECOL...,INSTITUCIÓN PRIVADA,65.034.388-3,Agrupaciones,"Microempresa (microcrédito, apoyo, fomento)",9010541 otorgada por INSPECCION PROVINCIAL DEL...,$ 1,$ 1,$ 1,Representante legal\nROSA HERMINIA CANIULEN L...,SECTOR LILICURA S/N\n ...,01-09-2011 10:33,Institución Privada Sin Fines de Lucro


#### Internacional

Serán aquellos que tengan como tipo `Organismo Internacional`

In [ ]:
df_data.loc[df_data["Tipo institución"] == 'Organismo Internacional']

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
69500502,ORGANIZACION DE LAS NACIONES UNIDAS PARA LA AG...,INSTITUCIÓN PRIVADA,69.500.502-4,Organismo Internacional,"Desarrollo local, desarrollo comunal, urbano",0 otorgada por 0,$ 0,$ 0,$ 0,Representante legal\nIVAN ENRIQUE CHACÓN PALOM...,AV. DAG HAMMARSKJÖLD 3241\n ...,11-11-2021 15:47,None


Es la única organización internacional

In [ ]:
df_data.loc[df_data["Tipo institución"] == 'Organismo Internacional', "tipo_institucion_final"] = "Organismo Internacional"

#### Otros

Veamos si hay organizaciones sin tipo_institucion_final.

In [ ]:
df_data[df_data["tipo_institucion_final"].isnull()]

,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final


Están todas listas :)

### Más procesamiento

Hay dos cosas que debemos hacer antes de continuar:
1. Eliminar la fila con el rut 55555555. Este RUT es genérico, y de hecho el nombre en esa fila es un nombre de prueba, de cuando estaban probando el sistema al parecer.
2. Eliminar caracteres extraños dentro de los nombres de las instituciones.

In [ ]:
df_data.loc["55555555"]

Nombre                            PRUEBA DE INSCRIPCIÓN INSCRIPCIÓN EN LA LEY
Tipo 1                                                    INSTITUCIÓN PRIVADA
RUT                                                              55.555.555-5
Tipo institución                                                          ONG
Área temática                             Informática (computación, internet)
Personalidad jurídica                                      34 otorgada por Mi
Patrimonio                                                                $ 0
Capital social                                                            $ 0
Estado resultado                                                          $ 0
Directorio                  Representante legal\nYASNA CAROLINA ROJAS RIVE...
Sucursales                  SALITRERA SEBASTOPOL 2044\n                   ...
Inscripción en Registros                                     23-09-2005 15:56
tipo_institucion_final                 Institución Privada Sin F

In [ ]:
df_data.drop("55555555", inplace=True)
df_data


,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final
65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,Institución Privada Sin Fines de Lucro
65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,Institución Privada Sin Fines de Lucro
65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,Institución Privada Sin Fines de Lucro
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,Institución Privada Sin Fines de Lucro
77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43,Empresa
4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,Institución Privada Sin Fines de Lucro
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,Institución Privada Sin Fines de Lucro
77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10,Empresa
65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,Institución Privada Sin Fines de Lucro
77546786,REPLACE 360 SPA,INSTITUCIÓN PRIVADA,77.546.786-K,Empresas,Salud,23691 otorgada por Registro de Comercio de San...,$ 1 000 000,$ 1 000 000,$ 1 000 000,Representante legal\nMARCELO CARLOS MARCONI TO...,LAS ARAUCARIAS 1733\n421\n ...,10-08-2022 23:57,Empresa


In [ ]:
df_data["Nombre normalizado"] = df_data["Nombre"].apply(unidecode)
df_data.loc["69500502", "Nombre"], df_data.loc["69500502", "Nombre normalizado"]

('ORGANIZACION DE LAS NACIONES UNIDAS PARA\xa0LA AGRICULTURA Y LA',
 'ORGANIZACION DE LAS NACIONES UNIDAS PARA LA AGRICULTURA Y LA')

Parece funcionar bien. Revisemos los casos donde `Nombre` es distinto a `Nombre normalizado`

In [ ]:
df_data.loc[df_data["Nombre"] != df_data["Nombre normalizado"], ["Nombre", "Nombre normalizado"]]

,Nombre,Nombre normalizado
65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,FUNDACION DE BENEFICIENCIA ENDEMICA
65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,FUNDACION PARA LA INVESTIGACION DEL CUIDADO
77517649,INGENIERÍA ROVISEN SPA,INGENIERIA ROVISEN SPA
69500502,ORGANIZACION DE LAS NACIONES UNIDAS PARA LA AG...,ORGANIZACION DE LAS NACIONES UNIDAS PARA LA AG...
65173837,UNIÓN COMUNAL DE COMITÉS DE AGUA POTABLE RURAL...,UNION COMUNAL DE COMITES DE AGUA POTABLE RURAL...
65090383,COMITÉ OVINO COMUNA LANCO,COMITE OVINO COMUNA LANCO
65211227,CORPORACIÓN PARA EL DESARROLLO DEL MAR,CORPORACION PARA EL DESARROLLO DEL MAR
65207445,CORPORACIÓN CENTRO NACIONAL DE INTELIGENCIA AR...,CORPORACION CENTRO NACIONAL DE INTELIGENCIA AR...
65151574,AGRUPACION DE PEQUEÑOS GANADEROS EL VALLE,AGRUPACION DE PEQUENOS GANADEROS EL VALLE
70037600,FUNDACIÓN CIUDAD DEL NIÑO,FUNDACION CIUDAD DEL NINO


# Unificar datos desde ambas fuentes

In [ ]:
df_data = df_data.reset_index() # Ahora la columna index contiene el rut sin dígito verificador
df_data

,index,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final,Nombre normalizado
0,65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,Institución Privada Sin Fines de Lucro,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...
1,65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,Institución Privada Sin Fines de Lucro,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...
2,65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,Institución Privada Sin Fines de Lucro,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...
3,65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,Institución Privada Sin Fines de Lucro,FUNDACION DE BENEFICIENCIA ENDEMICA
4,77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43,Empresa,INVERSKIN SPA
5,4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,Institución Privada Sin Fines de Lucro,ERCIARIO BERNARDO SILVA CUADRA
6,65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,Institución Privada Sin Fines de Lucro,FUNDACION PARA LA INVESTIGACION DEL CUIDADO
7,77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10,Empresa,INGENIERIA ROVISEN SPA
8,65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,Institución Privada Sin Fines de Lucro,AGRUPACION LUPUS CHILE
9,77546786,REPLACE 360 SPA,INSTITUCIÓN PRIVADA,77.546.786-K,Empresas,Salud,23691 otorgada por Registro de Comercio de San...,$ 1 000 000,$ 1 000 000,$ 1 000 000,Representante legal\nMARCELO CARLOS MARCONI TO...,LAS ARAUCARIAS 1733\n421\n ...,10-08-2022 23:57,Empresa,REPLACE 360 SPA


In [ ]:
df_data["rut_dv"] = df_data["RUT"].str.replace(".", "")
df_data["rut_dv"]

<ipython-input-54-4a9854a70239>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_data["rut_dv"] = df_data["RUT"].str.replace(".", "")


0     65160115-0
1     65170251-8
2     65667480-6
3     65188109-9
4     77377774-8
5      4860556-7
6     65205548-6
7     77517649-0
8     65001733-1
9     77546786-K
10    65123393-3
11    69500502-4
12    65034388-3
13    65064656-8
14    65189901-K
15    65542390-7
16    65212859-9
17    65173837-7
18    10105973-1
19    65059612-9
20    65205332-7
21    77542064-2
22    72299300-4
23    65090383-8
24    65211227-7
25    77328575-6
26    65209800-2
27    65207445-6
28    65151574-2
29    77371633-1
30    77517391-2
31    70037600-1
32    77374642-7
33    65035014-6
34    65124334-3
35    72847600-1
36    65454820-K
37    72319400-8
38    77534693-0
39    65184889-K
40    65936650-9
41    65207282-8
42    65016972-7
43    65904370-K
44    77547348-7
45    61602189-3
46    65157826-4
47    77419201-8
Name: rut_dv, dtype: object

In [ ]:
df_data["empresa_tipo"] = None
df_data["empresa_subtipo"] = None
df_data["provincia"] = None
df_data["comuna"] = None
df_data

,index,Nombre,Tipo 1,RUT,Tipo institución,Área temática,Personalidad jurídica,Patrimonio,Capital social,Estado resultado,Directorio,Sucursales,Inscripción en Registros,tipo_institucion_final,Nombre normalizado,rut_dv,empresa_tipo,empresa_subtipo,provincia,comuna
0,65160115,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,INSTITUCIÓN PRIVADA,65.160.115-0,Asociaciones,Economía y comercio,275-2 otorgada por SUBSECRETARÍA DE ECONOMÍA Y...,$ 0,$ 0,$ 0,Secretario(a)\nAIDA ESPEJO GALLEGUILLOS \n16....,ROSARIO 1541\n ...,06-11-2022 12:30,Institución Privada Sin Fines de Lucro,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,65160115-0,None,None,None,None
1,65170251,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,INSTITUCIÓN PRIVADA,65.170.251-8,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Educación,278697 otorgada por Registro Civil,$ 1 500 000,$ 1 000 000,$ 1 000 000,Tesorero(a)\nBASRHIAN PAUL CARRASCO CAMPOS\n1...,MANUEL DE SALAS 35\n ...,08-01-2019 21:26,Institución Privada Sin Fines de Lucro,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,65170251-8,None,None,None,None
2,65667480,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,INSTITUCIÓN PRIVADA,65.667.480-6,"Corporaciones (Ej: Educación, Salud, Cultura, ...","Cultura (teatro, música, danza, artes)",003/2006 otorgada por Ilustre Municipalidad de...,$ 45 372 695,$ 45 372 695,$ 0,Representante legal\nOSVALDO DE JESUS ROJAS MO...,AVDA. O´HIGGINS S/N\n ...,19-06-2006 10:50,Institución Privada Sin Fines de Lucro,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,65667480-6,None,None,None,None
3,65188109,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,INSTITUCIÓN PRIVADA,65.188.109-9,"Fundaciones (Ej: CEMA Chile, Integra, Cruz Roj...",Otros,304327 otorgada por Ilustre Municipalidad de J...,$ 300 000,$ 300 000,$ 150 000,Representante legal\nCAROLINA BUSTAMANTE DEL S...,LA PÓLVORA 447\n ...,06-03-2020 16:38,Institución Privada Sin Fines de Lucro,FUNDACION DE BENEFICIENCIA ENDEMICA,65188109-9,None,None,None,None
4,77377774,INVERSKIN SPA,INSTITUCIÓN PRIVADA,77.377.774-8,Empresas,Hombres,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,DESCONOCIDA S/N\n ...,23-11-2021 08:43,Empresa,INVERSKIN SPA,77377774-8,None,None,None,None
5,4860556,ERCIARIO BERNARDO SILVA CUADRA,INSTITUCIÓN PRIVADA,4.860.556-7,Agrupaciones,Apoyo y fomento productivo,S/I otorgada por S/I,$ 0,$ 0,$ 0,NaN,SANTIAGO,28-06-2007 02:04,Institución Privada Sin Fines de Lucro,ERCIARIO BERNARDO SILVA CUADRA,4860556-7,None,None,None,None
6,65205548,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,INSTITUCIÓN PRIVADA,65.205.548-6,"Centros (Ej: Madres, Padres, Apoderados, Alumn...",Educación,3454-2022. otorgada por Notario Público de San...,$ 700 000,$ 700 000,$ 700 000,Representante legal\nCLAUDIA ANDREA MIRANDA CA...,CAMINO DE LA POSADA 13455\n ...,11-10-2022 17:21,Institución Privada Sin Fines de Lucro,FUNDACION PARA LA INVESTIGACION DEL CUIDADO,65205548-6,None,None,None,None
7,77517649,INGENIERÍA ROVISEN SPA,INSTITUCIÓN PRIVADA,77.517.649-0,Empresas,"Informática (computación, internet)",77.517.649-0 otorgada por Registro de Empresas...,$ 2 000 000,$ 1 000 000,$ 1 500 000,Director(a)\nJOSÉ MIGUEL YÁÑEZ ARANCIBIA\n15.9...,HUÉRFANOS 1055\n503\n ...,06-11-2022 14:10,Empresa,INGENIERIA ROVISEN SPA,77517649-0,None,None,None,None
8,65001733,AGRUPACION LUPUS CHILE,INSTITUCIÓN PRIVADA,65.001.733-1,Agrupaciones,Responsabilidad Social (empresas y comunidad),1104 otorgada por Municipalidad de Puente Alto,$ 0,$ 0,$ 0,Representante legal\nERIKA MARLENE LUTZ TORO\n...,AV. CONCHA Y TORO 3459\n ...,13-04-2009 23:52,Institución Privada Sin Fines de Lucro,AGRUPACION LUPUS CHILE,65001733-1,None,None,None,None
9,77546786,REPLACE 360 SPA,INSTITUCIÓN PRIVADA,77.546.786-K,Empresas,Salud,23691 otorgada por Registro de Comercio de San...,$ 1 000 000,$ 1 000 000,$ 1 000 000,Representante legal\nMARCELO CARLOS MARCONI TO...,LAS ARAUCARIAS 1733\n421\n ...,10-08-2022 23:57,Empresa,REPLACE 360 SPA,77546786-K,None,None,None,None


In [ ]:
df_data_final = df_data[["index", "rut_dv", "Nombre", "empresa_tipo", "empresa_subtipo", "provincia", "comuna", "tipo_institucion_final"]].rename(columns={"index":"rut", "Nombre": "nombre"})
df_data_final

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,65160115,65160115-0,ASOCIACION DE INDUSTRIALES DE TOCOPILLA ASINTO...,None,None,None,None,Institución Privada Sin Fines de Lucro
1,65170251,65170251-8,INTERNATIONAL INSTITUTE FOR PHILOSOPHY AND SOC...,None,None,None,None,Institución Privada Sin Fines de Lucro
2,65667480,65667480-6,MUSEO DE HISTORIA NATURAL Y CULTURAL DEL DESIE...,None,None,None,None,Institución Privada Sin Fines de Lucro
3,65188109,65188109-9,FUNDACIÓN DE BENEFICIENCIA ENDÉMICA,None,None,None,None,Institución Privada Sin Fines de Lucro
4,77377774,77377774-8,INVERSKIN SPA,None,None,None,None,Empresa
5,4860556,4860556-7,ERCIARIO BERNARDO SILVA CUADRA,None,None,None,None,Institución Privada Sin Fines de Lucro
6,65205548,65205548-6,FUNDACIÓN PARA LA INVESTIGACIÓN DEL CUIDADO,None,None,None,None,Institución Privada Sin Fines de Lucro
7,77517649,77517649-0,INGENIERÍA ROVISEN SPA,None,None,None,None,Empresa
8,65001733,65001733-1,AGRUPACION LUPUS CHILE,None,None,None,None,Institución Privada Sin Fines de Lucro
9,77546786,77546786-K,REPLACE 360 SPA,None,None,None,None,Empresa


In [ ]:
df_merged = pd.concat([df_final, df_data_final], ignore_index=True)
df_merged

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,60908000,60908000-0,JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,Estado
1,65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,Institución Privada Sin Fines de Lucro
2,81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,Institución de Educación Superior
3,65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,Institución Privada Sin Fines de Lucro
4,69508200,69508200-2,ORGANIZACIÓN DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,Internacional
...,...,...,...,...,...,...,...,...
1735,65904370,65904370-K,AGRUPACION PATAGONIA VISUAL,None,None,None,None,Institución Privada Sin Fines de Lucro
1736,77547348,77547348-7,SOCIEDAD COMERCIAL COLORIS BIOTECH SPA,None,None,None,None,Empresa
1737,61602189,61602189-3,HOSPITAL GUILLERMO GRANT BENAVENTE,None,None,None,None,Estado
1738,65157826,65157826-4,CORPORACION NACIONAL DE DIFUSION DE LAS CIENCI...,None,None,None,None,Institución Privada Sin Fines de Lucro


In [ ]:
df_merged["rut_dv"] = df_merged["rut_dv"].str.lower()
df_merged["nombre"] = df_merged["nombre"].apply(unidecode)
df_merged

,rut,rut_dv,nombre,empresa_tipo,empresa_subtipo,provincia,comuna,tipo_institucion_final
0,60908000,60908000-0,JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECA,INSTITUCIONES FISCALES,ORG. ADMINISTRACION PUBLICA,SANTIAGO,SANTIAGO,Estado
1,65937660,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,ORG. SIN FINES DE LUCRO,CORPORACION,ANTOFAGASTA,ANTOFAGASTA,Institución Privada Sin Fines de Lucro
2,81698900,81698900-0,PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE,PERSONA JURIDICA COMERCIAL,SOC. RESPONSABILIDAD LIMITADA,SANTIAGO,SANTIAGO,Institución de Educación Superior
3,65072516,65072516-6,CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUD...,ORG. SIN FINES DE LUCRO,CORPORACION,VALPARAÍSO,VALPARAISO,Institución Privada Sin Fines de Lucro
4,69508200,69508200-2,ORGANIZACION DE ESTADOS IBEROAMERICANOS PARA L...,ORGANISMOS INTERNACIONALES,ORGANISMOS INTERNACIONALES,SANTIAGO,PROVIDENCIA,Internacional
...,...,...,...,...,...,...,...,...
1735,65904370,65904370-k,AGRUPACION PATAGONIA VISUAL,None,None,None,None,Institución Privada Sin Fines de Lucro
1736,77547348,77547348-7,SOCIEDAD COMERCIAL COLORIS BIOTECH SPA,None,None,None,None,Empresa
1737,61602189,61602189-3,HOSPITAL GUILLERMO GRANT BENAVENTE,None,None,None,None,Estado
1738,65157826,65157826-4,CORPORACION NACIONAL DE DIFUSION DE LAS CIENCI...,None,None,None,None,Institución Privada Sin Fines de Lucro


In [ ]:
df_merged.to_excel(os.path.join(results_dir, "data_instituciones_anid.xlsx"), index=False)